2. Write a generator function that generates prime numbers via the
[Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).

In [1]:
test_cases = (
    (10, (2, 3, 5, 7)),
    (11, (2, 3, 5, 7, 11)),
    (12, (2, 3, 5, 7, 11)),
    (13, (2, 3, 5, 7, 11, 13)),
)

def test():
    for n, known_good_output in test_cases:
        assert tuple(sorted(primes(n))) == known_good_output

In [2]:
n = 10**6

This cell and the three following cells were added
during the 2016-06-27 COhPy meeting
to show and discuss the extra indentation
incurred by not using a continue statement.

In [3]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n+1)]
    for i in range(2, n):
        if is_prime[i]:
            yield i
            for j in range(2*i, n, i):
                is_prime[j] = False

test()

In [4]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 546 ms per loop


78498

The following cell uses a continue statement
to avoid indenting following code.
Code that has nested tests often benefits
by using continue statements.

In [5]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n+1)]
    for i in range(2, n):
        if not is_prime[i]:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            is_prime[j] = False

test()

In [6]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 563 ms per loop


78498

Next I try using a dictionary to hold the sieve. It is much slower than using a list.

In [7]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = {i:True for i in range(2, n)}
    for i in range(2, n):
        if not is_prime[i]:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            is_prime[j] = False

test()

In [8]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 996 ms per loop


78498

Try another way using dictionaries.
This way deletes values from the sieve after they will no longer be used.
This is slower yet.

In [9]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = {i:True for i in range(2, n)}
    for i in range(2, n):
        if not is_prime[i]:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            is_prime[j] = False
        del is_prime[i]

test()

In [10]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 986 ms per loop


78498

Next I try a using a set to hold the sieve.
The sieve only holds the numbers that are not prime.
Since the prime numbers are not in the set,
this technique might not be the classic sieve.
It is still much slower than using a list.

In [11]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    composites = set()
    for i in range(2, n):
        if i in composites:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            composites.add(j)

test()

In [12]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 951 ms per loop


78498

This time I try lists again, with a modest optimization of not iterating over the even numbers greater than two. It is a little bit faster than iterating through all the numbers.

In [13]:
from itertools import islice, count, chain

# n = 20
# list(chain([2], range(3, n+1, 2)))

In [14]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n)]
    for i in chain([2], range(3, n, 2)):
        if not is_prime[i]:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            is_prime[j] = False

test()

In [15]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 501 ms per loop


78498

Added after 2016-06-27: 

Andrew Kubera's code uses square root optimization.
I normally use square root optimization for
prime number calculations, so I was surprised
that it worked with SoE, and surprised that I was surprised.
Of course it works.
So I add it, first to the simple range(2, n) version,
then to the chain([2], range(3, n, 2)) version.

In [16]:
from math import sqrt, floor

In [17]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n+1)]
    stop = int(floor(sqrt(n))) + 1
    for i in range(2, stop):
        if is_prime[i]:
            yield i
            for j in range(2*i, n, i):
                is_prime[j] = False

    for i in range(stop, n):
        if is_prime[i]:
            yield i

test()

In [18]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 398 ms per loop


78498

In [19]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n)]
    stop = int(floor(sqrt(n))) + 1
    for i in chain([2], range(3, stop, 2)):
        if not is_prime[i]:
            continue
        # i is prime
        yield i
        for j in range(2*i, n, i):
            is_prime[j] = False

    for i in range(stop, n):
        if is_prime[i]:
            yield i

test()

In [20]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 397 ms per loop


78498

Now to try to play with some functional stuff.

First try a generator expression (maybe not very functional).

In [21]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n+1)]
    stop = int(floor(sqrt(n))) + 1
    for i in range(2, stop):
        if is_prime[i]:
            yield i
            for j in range(2*i, n, i):
                is_prime[j] = False

    yield from (i for i in range(stop, n) if is_prime[i])

test()

In [22]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 414 ms per loop


78498

In [23]:
def primes(n):
    '''Generates primes less than or equal to n.
    
    Uses Sieve of Eratosthenes.'''
    n += 1
    is_prime = [True for _ in range(n+1)]
    stop = int(floor(sqrt(n))) + 1
    for i in range(2, stop):
        if is_prime[i]:
            yield i
            for j in range(2*i, n, i):
                is_prime[j] = False

    # .__getitem__ is from Kubera.
    yield from filter(is_prime.__getitem__, range(stop, n))

test()

In [24]:
%timeit len(tuple(primes(n)))
len(tuple(primes(n)))

1 loop, best of 3: 405 ms per loop


78498

Looks like cell #17 was the best for speed and readability.